#### AI/ML directed high-throughput virtual screening of linear probes

#### Siva Dasetty, Maximilian Topel | The Ferguson Lab, Pritzker School of Molecular Engineering, U.Chicago

## imports

In [9]:
import os
import os.path
import sys
import importlib
import yaml

import subprocess

from time import time
from datetime import datetime

import numpy as np
import pandas as pd
import math

import scipy

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import Descriptors
from rdkit.Chem.Fragments import *

from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from matplotlib import rcParams
import matplotlib as mpl
from matplotlib  import cm
from PIL import Image
from matplotlib import colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
from matplotlib.cbook import get_sample_data

import multiprocessing

from sklearn.metrics import mean_squared_error

sys.path.append("/home/sivadasetty/scratch-midway2/pfas/analysis/NB_PFAS/activeLearning/codes/")


import utils_activeLearning_Oliver as ut

import utils_functions as utils_functions
import utils_figures as utils_figures

importlib.reload(ut)
importlib.reload(utils_functions)
importlib.reload(utils_figures)

import selfies as sf

from oliver_vae.VAE import VAE_encode, VAE_decode

import torch
from torch import nn

import copy

## plot settings

In [10]:
plt.rcParams.update({'font.size': 20})

# time stamp
now = datetime.now()
now = str(now)[:10]
figures_dir = './Figures/{}'.format(str(now)).replace(':','-')

## read all search molecules

In [11]:
# read all potential linear probes
trainingfolder = '/home/sivadasetty/scratch-midway2/pfas/analysis/Oliver_trainingData_latentspace'
all_potential_linear_probes = pd.read_csv(trainingfolder + '/SMILES_linearProbes_All_Properties.csv')

# extract smiles
all_potential_linear_probesList = all_potential_linear_probes.smiles

## generate dictionary of smiles and index in search set

In [12]:
dict_smiles_all_potential_linear_probes = dict(zip(all_potential_linear_probes.smiles, all_potential_linear_probes.index))

## read saved PCA arrays of search set and molecules in ZINC trainings et

In [13]:
## read saved PCA arrays
projected_all_search_linear_probes_array = np.load('/home/sivadasetty/scratch-midway2/pfas/analysis/Oliver_trainingData_latentspace/PCA_AllSearchLinearProbes.npy')

projected_zinc_set = np.load('/home/sivadasetty/scratch-midway2/pfas/analysis/Oliver_trainingData_latentspace/PCA_ZINC_TrainingSet.npy')


## read completed dataset

In [14]:
#Load canonicalized SMILES of all linear probes
simdir = '/media/sivadasetty/Seagate Desktop Drive/pfas/'

simdir = '/home/sivadasetty/scratch-midway2/pfas/'


deltaGfile = simdir + '/analysis/NB_PFAS/WeeklyAnalysis/deltaG-data/wsmiles_deltaG-linearProbesBlockArea-b3-pbmd-4cvs.txt'
deltaGdata = pd.read_csv(deltaGfile, delim_whitespace=True, names=["Method", "Analyte", "lprobe", 
                                                                   "lflourines", "lhydrogen", "hhydrogen", 
                                                                   "hcarbons", "Probe", 
                                                                   "Avg", "SD", "smiles", "formula"], comment="#")

#print(deltaGdata.Probe.values)

sdsfldata = deltaGdata[ (deltaGdata.Analyte == "SDS")
                      & (deltaGdata.Method == "PBMD-4CVS-GPU") ]
#sdsfldata = sdsfldata.sort_values(by=['lprobe', 'lflourines'])

pfosfldata = deltaGdata[ (deltaGdata.Analyte == "PFOSD") 
                       & (deltaGdata.Method == "PBMD-4CVS-GPU") ]

pfosfldataCopy = pfosfldata.copy(deep=False)
#pfosfldata = pfosfldata.sort_values(by=['lprobe', 'lflourines'])

# for ratio measure
#yflerror = (pfosfldata.Avg.values/sdsfldata.Avg.values)*np.sqrt( (sdsfldata.SD.values/sdsfldata.Avg.values)**2 +  (pfosfldata.SD.values/pfosfldata.Avg.values)**2 )
#pfosfldataCopy['selectivity'] = pfosfldata.Avg.values/sdsfldata.Avg.values

# for delta delta G measure
pfosfldataCopy['selectivity'] = pfosfldata.Avg.values - sdsfldata.Avg.values
yflerror = np.sqrt( (sdsfldata.SD.values)**2 +  (pfosfldata.SD.values)**2 )
pfosfldataCopy['selectivityError'] = yflerror


# for sensitivity
pfosfldataCopy['sensitivity'] = pfosfldata.Avg.values
pfosfldataCopy['sensitivityError'] = pfosfldata.SD.values

    
# filter probes using alphabet list extracted from the training set molecules
alphabet_list = pd.read_csv('codes/data/alphabet-list.csv')['alphabets'].tolist()
#print(alphabet_list)

alpha_exists = ut.check_molecules_alphabets_exists(pfosfldataCopy)

#print("\n probes alphabet list")
#print(alpha_exists)
#print(np.shape(alpha_exists))

pfosfldataCopy['smilesValid'] = np.array(alpha_exists)

pfosfldataCopy = pfosfldataCopy[ (pfosfldataCopy.smilesValid == True) ]

# filter for C < 15
pfosfldataCopy = pfosfldataCopy[ pfosfldataCopy['lprobe'] < 15 ]

# sort by ncarbons
#pfosfldataCopy = pfosfldataCopy.sort_values(by=['lprobe']) #, 'lflourines'])

pfosfldataCopy

,Method,Analyte,lprobe,lflourines,lhydrogen,hhydrogen,hcarbons,Probe,Avg,SD,smiles,formula,selectivity,selectivityError,sensitivity,sensitivityError,smilesValid
0,PBMD-4CVS-GPU,PFOSD,4,0,10,-1,-1,C4/F0,-0.501825,0.534080,CCCC,C4H10,0.124415,0.541591,-0.501825,0.534080,True
1,PBMD-4CVS-GPU,PFOSD,6,0,14,-1,-1,C6/F0,-1.819519,0.640062,CCCCCC,C6H14,-0.040476,0.679777,-1.819519,0.640062,True
2,PBMD-4CVS-GPU,PFOSD,8,0,18,-1,-1,C8/F0,-2.620010,0.264206,CCCCCCCC,C8H18,1.196455,0.290748,-2.620010,0.264206,True
3,PBMD-4CVS-GPU,PFOSD,10,0,22,-1,-1,C10/F0,-3.852597,0.212905,CCCCCCCCCC,C10H22,0.595865,0.240479,-3.852597,0.212905,True
4,PBMD-4CVS-GPU,PFOSD,12,0,26,-1,-1,C12/F0,-4.290535,0.607122,CCCCCCCCCCCC,C12H26,1.676133,0.666850,-4.290535,0.607122,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,PBMD-4CVS-GPU,PFOSD,9,0,20,2,0,C9Br0/NH2,-4.578463,0.088387,CCCC(Br)(Br)C(Br)(Br)C(Br)(Br)C(Br)(Br)C(Br)(B...,C9H9Br12N,0.693603,0.232540,-4.578463,0.088387,True
519,PBMD-4CVS-GPU,PFOSD,3,3,11,0,3,C3F3/N(CH3)3,-1.359146,0.327960,C[N+](C)(C)CCC(F)(F)F,C6H13F3N,-0.042718,0.408764,-1.359146,0.327960,True
522,PBMD-4CVS-GPU,PFOSD,12,25,5,0,2,C12Cl25/P(CH3)2,-7.310449,0.631328,CP(C)C(Cl)(Cl)C(Cl)(Cl)C(Cl)(Cl)C(Cl)(Cl)C(Cl)...,C14H6Cl25P,5.527516,1.097425,-7.310449,0.631328,True
523,PBMD-4CVS-GPU,PFOSD,7,15,5,0,2,C7F15/N(CH3)2,-4.964294,0.862647,CN(C)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F...,C9H6F15N,-0.463810,0.935841,-4.964294,0.862647,True


## find indices of completed probes

In [15]:
index_completed_probes = []
for key in pfosfldataCopy.smiles.values.tolist():
    index_completed_probes.append( dict_smiles_all_potential_linear_probes.get(key) )


## separate PCs and smiles of completed and unexplored probes using indices

In [16]:
pc_completed = projected_all_search_linear_probes_array[index_completed_probes]
pc_unexplored = np.delete(projected_all_search_linear_probes_array, index_completed_probes, axis=0)

smiles_completed = all_potential_linear_probes.iloc[index_completed_probes]
smiles_unexplored = all_potential_linear_probes[~all_potential_linear_probes.index.isin(index_completed_probes)]

# check smiles extracted and read are the same in the dataframes

In [17]:
# two identical dataframes
df1 = smiles_completed[['smiles']].reset_index(drop=True)
df2 = pfosfldataCopy[['smiles']].reset_index(drop=True)

# check if both are equal
print(df1.equals(df2))

True


## set device

In [18]:
torch.cuda.empty_cache()
n_devices = torch.cuda.device_count()
print('Planning to run on {} GPUs.'.format(n_devices-1))

Planning to run on 3 GPUs.


## prepare data for training

In [19]:
npcs = 5 #select number of PCs
pcListNames=[]
for (item1, item2) in zip(['pc']*npcs, np.arange(0,npcs)):
    pcListNames.append(item1+str(item2))

trainData = pd.DataFrame( pc_completed[:,np.arange(0,npcs)], columns=pcListNames, dtype = 'float' )

ysd = pfosfldataCopy.selectivityError.values
y = pfosfldataCopy.selectivity.values

trainData['f_obj_selectivity'] = y * -1 # for maximizing 
trainData['f_obj_error_selectivity'] = ysd

ysd = pfosfldataCopy.sensitivityError.values
y = pfosfldataCopy.sensitivity.values

trainData['f_obj_sensitivity'] = y * -1 # for maximizing
trainData['f_obj_error_sensitivity'] = ysd

print( trainData.f_obj_selectivity.min() )
print( trainData.f_obj_selectivity.max() )

print( trainData.f_obj_sensitivity.min() )
print( trainData.f_obj_sensitivity.max() )

-9.435065792882614
3.100481351149976
-0.9333558445990184
9.794834454266525


## multi-objective Bayesian optimization using random scalarizations

In [21]:
multi_trainData = trainData.copy()
multi_trainData

,pc0,pc1,pc2,pc3,pc4,f_obj_selectivity,f_obj_error_selectivity,f_obj_sensitivity,f_obj_error_sensitivity
0,-49.518326,9.005636,-0.347310,-3.799499,12.873461,-0.124415,0.541591,0.501825,0.534080
1,-49.171539,9.326957,0.078968,-3.641393,12.023334,0.040476,0.679777,1.819519,0.640062
2,-49.430145,8.978824,0.882766,-3.796247,11.605146,-1.196455,0.290748,2.620010,0.264206
3,-47.139896,7.172888,2.530454,-3.327389,9.268016,-0.595865,0.240479,3.852597,0.212905
4,-44.073818,5.382994,4.331920,-1.006031,5.727169,-1.676133,0.666850,4.290535,0.607122
...,...,...,...,...,...,...,...,...,...
247,3.287469,-5.507845,-3.342284,0.881792,1.544212,-0.693603,0.232540,4.578463,0.088387
248,-34.426353,1.413089,1.311949,1.265554,-3.103536,0.042718,0.408764,1.359146,0.327960
249,67.572304,-1.427993,0.189692,6.350353,0.993674,-5.527516,1.097425,7.310449,0.631328
250,20.535923,-3.621423,3.692337,-4.924819,-0.820699,0.463810,0.935841,4.964294,0.862647


In [19]:
import warnings
warnings.filterwarnings("ignore")

npcs=5

# random scalarization
niter=0
q = 20 # number of candidates
selected_probes = list()
selected_index = list()

while (len(selected_probes) < q):
    
    selectivityData = utils_functions.trainGPR(multi_trainData, 'selectivity', npcs, isotropic=0)
    sensitivityData = utils_functions.trainGPR(multi_trainData, 'sensitivity', npcs, isotropic=0)
    
    gprModel_selectivity =  selectivityData['gprModel']
    gprModel_sensitivity =  sensitivityData['gprModel']
    
    X_selectivity = selectivityData['X']
    y_selectivity = selectivityData['y']
    ysdplot_selectivity = selectivityData['ysdplot']
    y_sensitivity = sensitivityData['y']
    ysdplot_sensitivity = sensitivityData['ysdplot']
    
    theta = utils_functions.get_scalarization()
    kappa = utils_functions.loguniform(low=np.log(1e-4), high=np.log(10000))

    # search set
    x = pc_unexplored[:,0:npcs]
    
    # EI acq [enable for EI]
    #ei_selectivity = utils_functions.expected_improvement(x, gprModel_selectivity, y_selectivity, greater_is_better=False, n_params=npcs)
    #ei_sensitivity = utils_functions.expected_improvement(x, gprModel_sensitivity, y_sensitivity, greater_is_better=False, n_params=npcs)
    
    # predict values
    y_pred_selectivity, y_pred_std_selectivity = gprModel_selectivity.predict(x,  return_std=True)
    y_pred_sensitivity, y_pred_std_sensitivity = gprModel_sensitivity.predict(x,  return_std=True)
    
    # UCB acq
    ucb_selectivity = utils_functions.ucb(y_pred_selectivity.ravel(), y_pred_std_selectivity, kappa)
    ucb_sensitivity = utils_functions.ucb(y_pred_sensitivity.ravel(), y_pred_std_selectivity, kappa)
        
    # random scalarizations
    aq = utils_functions.scalarize(ucb_selectivity, ucb_sensitivity, theta).flatten()
    
    selected_name = smiles_unexplored.smiles.values[aq.argmax()]
    
    #print("iter")
    #print(selected_name)
    # update training set
    
    if selected_name not in selected_probes:
        
        selected_probes.append(selected_name)
        selected_index.append(aq.argmax())
        
        print(f'Selected probe #{len(selected_probes)}/{q}: {selected_name}')
        
        #################################################################################################################
        # ADD IDENTIFIED PROBE AND ITS PREDICTED RESPONSE TO TRAINING SET FOR NEXT ITERATION [KRIGING BELIEVER APPROACH]        
        Xnew = pc_unexplored[aq.argmax()][0:npcs]
        
        # updated set
        Xup = np.vstack((X_selectivity, Xnew))    
        yup_selectivity = np.append(y_selectivity, y_pred_selectivity[aq.argmax()]).reshape(-1,1) 
        yup_sd_selectivity = np.append(ysdplot_selectivity, y_pred_std_selectivity[aq.argmax()]).reshape(-1,1)
        yup_sensitivity = np.append(y_sensitivity, y_pred_sensitivity[aq.argmax()]).reshape(-1,1) 
        yup_sd_sensitivity = np.append(ysdplot_sensitivity, y_pred_std_sensitivity[aq.argmax()]).reshape(-1,1)

        pcListNames=[]
        for (item1, item2) in zip(['pc']*npcs, np.arange(0,npcs)):
            pcListNames.append(item1+str(item2))
        
        columnNames = pcListNames + ['f_obj_selectivity', 'f_obj_error_selectivity',
                                     'f_obj_sensitivity', 'f_obj_error_sensitivity'] 
        
        # update data frame
        multi_trainData = pd.DataFrame(np.column_stack((Xup, 
                                                        yup_selectivity,
                                                        yup_sd_selectivity,
                                                        yup_sensitivity,
                                                        yup_sd_sensitivity)), columns=columnNames)

        print("iteration", niter)

    if niter > 1000:
        break
        
    niter += 1
    
selected_probes_df = pd.DataFrame(selected_probes, columns =['smiles']) 

Accuracy score for training data: 0.2072
R2 score for training data: 0.9217
[2.43601074e+01 2.00000000e+02 9.39990032e-01 6.70665475e-02
 1.59722354e+00]
Accuracy score for training data: 0.0697
R2 score for training data: 0.9832
[ 13.71718364 200.           0.34967344   5.48768455   0.59303081]
Selected probe #1/20: CNC(Br)(Br)C(Br)(Br)C(Br)(Br)C(Br)(Br)C(Br)(Br)C(Br)(Br)C(Br)(Br)C(C)(Br)Br
iteration 0
Accuracy score for training data: 0.2073
R2 score for training data: 0.9214
[2.43713456e+01 2.00000000e+02 9.39338962e-01 6.70527741e-02
 1.59647515e+00]
Accuracy score for training data: 0.0694
R2 score for training data: 0.9832
[ 13.80633268 200.           0.34877534   5.57453071   0.59482758]
Selected probe #2/20: CNCCCC(Cl)(Cl)C(Cl)(Cl)C(Cl)(Cl)C(Cl)(Cl)C(Cl)(Cl)C(Cl)(Cl)C(Cl)(Cl)C(Cl)(Cl)C(Cl)(Cl)Cl
iteration 1
Accuracy score for training data: 0.2116
R2 score for training data: 0.9194
[5.34595110e+00 6.32229673e-02 2.34007591e+00 4.24237973e+00
 2.00000000e+02]
Accuracy score for 

### Generate probe names

In [ ]:
probeNames = []
for i in range(selected_probes_df.shape[0]):
#for i in range(c8_12_hyd_fluor_amine_head_probes_df.shape[0]):
#for i in range(c3_fluorinated_amine_head_probes_df.shape[0]):

    smiles = selected_probes_df.smiles.values[i]
    #smiles = c8_12_hyd_fluor_amine_head_probes_df.smiles.values[i]
    #smiles = c3_fluorinated_amine_head_probes_df.smiles.values[i]
    
    label_i_c_count = smiles.count('C')
    
    label_i_f_count = smiles.count('F')
    
    label_i_cl_count = smiles.count('Cl')
    
    label_i_br_count = smiles.count('Br')
    
    if label_i_cl_count != 0:
        label_i_c_count -= label_i_cl_count
        
    fstring = '-f'
    if label_i_cl_count > 0:
        label_i_f_count = label_i_cl_count
        fstring = '-cl'
        
    if label_i_br_count > 0:
        label_i_f_count = label_i_br_count
        fstring = '-br'
        
        
     
    countP = -1
    if smiles.count('P') > 0:
        
        countP = smiles.count('P')
        smiles = smiles.replace('P', 'N')
        
    
    rdkProbe = Chem.MolFromSmiles(smiles, sanitize=True)
    
    head_index = None
    for atom in rdkProbe.GetAtoms():
        if atom.GetAtomicNum() == 7:
            head_index = atom.GetIdx()
        
    #print(head_index)
    
    label_i_head_group_h_count = -1
    label_i_head_group_c_count = -1
    
    if fr_NH2(rdkProbe) == 1 and fr_quatN(rdkProbe) == 0 :
        label_i_head_group_h_count = 2
        label_i_head_group_c_count = 0
        #print(probes[i], "primary amine")
    elif fr_NH1(rdkProbe) == 1 and fr_quatN(rdkProbe) == 0:
        label_i_head_group_h_count = 1
        label_i_head_group_c_count = 1
        #print(probes[i], "secondary amine")
    elif fr_NH0(rdkProbe) == 1 and fr_quatN(rdkProbe) == 0:
        label_i_head_group_h_count = 0
        label_i_head_group_c_count = 2
        #label_i_head_group = "0"
        #print(probes[i], "tertiary amine")
    elif fr_NH0(rdkProbe) == 1 and fr_quatN(rdkProbe) == 1:
        label_i_head_group_h_count = 0
        label_i_head_group_c_count = 3
        #label_i_head_group = ""
        #print(probes[i], "quaternary amine")
    elif fr_NH2(rdkProbe) == 1 and fr_quatN(rdkProbe) == 1 :
        label_i_head_group_h_count = 2
        label_i_head_group_c_count = 1
        #print(probes[i], "secondary amine cation") 
    elif fr_NH1(rdkProbe) == 1 and fr_quatN(rdkProbe) == 1:
        label_i_head_group_h_count = 1
        label_i_head_group_c_count = 2
        #print(probes[i], "tertiary amine cation")
    elif fr_NH0(rdkProbe) == 0 and fr_quatN(rdkProbe) == 1:
        label_i_head_group_h_count = 3
        label_i_head_group_c_count = 0

    if label_i_head_group_c_count > 0:
        label_i_c_count = label_i_c_count - label_i_head_group_c_count
    
    if countP == -1:
    
        if label_i_head_group_h_count == -1 and label_i_head_group_c_count == -1:
            label_i = 'c' + str(label_i_c_count) + fstring + str(label_i_f_count)
        elif label_i_head_group_h_count == 0:
            label_i = 'c' + str(label_i_c_count) + fstring + str(label_i_f_count) + '-n' + str(head_index) + '_ch3_' + str(label_i_head_group_c_count)
        elif label_i_head_group_c_count == 0:
            label_i = 'c' + str(label_i_c_count) + fstring + str(label_i_f_count) + '-n' + str(head_index) + '_h' + str(label_i_head_group_h_count)
        elif label_i_head_group_h_count > 0 and label_i_head_group_c_count > 0:
            label_i = 'c' + str(label_i_c_count) + fstring + str(label_i_f_count) + '-n' + str(head_index) + '_h' + str(label_i_head_group_h_count) + '_ch3_' + str(label_i_head_group_c_count)
    else:
        
        if label_i_head_group_h_count == -1 and label_i_head_group_c_count == -1:
            label_i = 'c' + str(label_i_c_count) + fstring + str(label_i_f_count)
        elif label_i_head_group_h_count == 0:
            label_i = 'c' + str(label_i_c_count) + fstring + str(label_i_f_count) + '-p' + str(head_index) + '_ch3_' + str(label_i_head_group_c_count)
        elif label_i_head_group_c_count == 0:
            label_i = 'c' + str(label_i_c_count) + fstring + str(label_i_f_count) + '-p' + str(head_index) + '_h' + str(label_i_head_group_h_count)
        elif label_i_head_group_h_count > 0 and label_i_head_group_c_count > 0:
            label_i = 'c' + str(label_i_c_count) + fstring + str(label_i_f_count) + '-p' + str(head_index) + '_h' + str(label_i_head_group_h_count) + '_ch3_' + str(label_i_head_group_c_count)
        
    
    probeName = label_i
    
    probeNames.append(probeName)
    print(probeName)

### save probes

In [21]:
selected_probes_df = pd.DataFrame(selected_probes, columns =['smiles']) 
selected_probes_df['name'] = probeNames

#c3_fluorinated_amine_head_probes_df['name'] = probeNames

target_dir = "/home/sivadasetty/scratch-midway2/pfas/probes/"
probe_dir = target_dir + '{}'.format(str(now)).replace(':','-')

selected_probes_df.to_csv(probe_dir + "-mobo-candidates.txt", 
                          header=None, index=None, sep='\t', mode='a')

#c3_fluorinated_amine_head_probes_df.to_csv(probe_dir + "-c3-head-group-candidates.txt", 
#                          header=None, index=None, sep='\t', mode='a')

### visualize if needed

In [ ]:
def mol_with_atom_index( mol ):
    atoms = mol.GetNumAtoms()
    for idx in range( atoms ):
        mol.GetAtomWithIdx( idx ).SetProp( 'molAtomMapNumber', str( mol.GetAtomWithIdx( idx ).GetIdx() ) )
    return mol

mol_with_atom_index( Chem.MolFromSmiles( selected_probes_df.smiles[0] ) )